In [500]:
import pandas as pd
import langdetect

In [501]:
#Load CSVs in dataframes
csv1 = pd.read_csv('1.csv', delimiter = ',',encoding = "ISO-8859-1")
csv2 = pd.read_csv('2.csv', delimiter = ',',encoding = "ISO-8859-1")



In [502]:
#Merge Dataframes
df = pd.concat([csv1,csv2])
df.reset_index(inplace=True, drop=True) 


In [503]:
#Filter by 'From: (Address)'
df_filtered_from = df[df['From: (Name)'].str.contains('***NAME OF THE PERSON TO FILTER***')]
#Filter NaN
df_filtered_nan = df_filtered_from[df_filtered_from['Body'].str.contains('NaN')==False]
df_filtered_nan = df_filtered_nan[df_filtered_nan['Body'].str.match('\r\n')==False]
df_filtered_nan = df_filtered_nan[df_filtered_nan['Body'].str.match(' \r\n\r\n')==False]
df_filtered_nan.reset_index(inplace=True, drop=True) 

In [504]:
#Create a list with languages for emails
language_list = []
for index, row in df_filtered_nan.iterrows():
    language_list.append(langdetect.detect(row['Body']))


In [505]:
#Add column to dataframe with languages
df_filtered_nan["Language"] = language_list
print(df_filtered_nan.shape)

(67, 20)


In [506]:
#Filter emails based on language
df_filtered_language = df_filtered_nan[df_filtered_nan['Language'].str.match('en')]

In [507]:
for index, row in df_filtered_language.iterrows():
    df_filtered_language.at[index, 'Body'] = row['Body'].split("****URL THAT USUALLY MEANS AN EMAIL END****")[0]
for index, row in df_filtered_language.iterrows():
    df_filtered_language.at[index, 'Body'] = row['Body'].split("_")[0]
for index, row in df_filtered_language.iterrows():
    df_filtered_language.at[index, 'Body'] = row['Body'].split("**** NAME OF THE PERSONA THAT MEANS EMAIL SIGNATURE *****")[0]

In [508]:
#Concatenate Body values in list
result = df_filtered_language['Body'].str.cat(sep=' ')
result = result.replace("\r","")
result = result.replace("\n","")
result = result.replace("_","")
result = result.replace(".","")
result = result.replace('****URL THAT USUALLY MEANS AN EMAIL END****',"")


                        

In [509]:
#Save string to txt
with open("Output.txt", "w", encoding="utf-8") as text_file:
    text_file.write(result)

In [510]:
#Write dataframe to csv for debugging purposes
df_filtered_language.to_csv("result.csv", sep=',')